# Proyect 1: DMRG and Particle in a Box

### To do:
1. fix problem with saving effective hamiltonians (memory problem change variables)to fix this create functions to clean up the code
2. Implement a rounding protocol 
3. create the basis generation funciton using itertools
4. Write a discretization algorithm for the particle in a box for open and periodic boundary conditions
5. Clean up code


In [1]:
import numpy as np
import scipy.linalg as la
from itertools import combinations, combinations_with_replacement


# 1.1 Density matrix computation

steps:

    1. Define Hamiltonian
    2. Choose system size and find the basis
    4. Compute the matrix elements
    5. Diagonalize the matrix
    6. Write Down the ground state as a matrix
    7. Compute the density matrix via matrix multiplication

### b) L = 4

Idea: Use the operators in matrix form to make the calculations

In [2]:
# definition of the operators
s_up = np.asarray([[0,1],[0,0]])
s_down = np.asarray([[0,0],[1,0]])
s_z = np.asarray([[1/2,0],[0,-1/2]])


In [3]:
def apply_operator(current_operator, next_operator,spin_chain, position):
    # applies the specified operators to the current and next place in the chain
    # args-> current_operator, next_operator: matrices representing the action on the current and 
    #        the next lattice site
    #         spin_chain: list of lists representing a basis vecotr
    #         position: index indicating the current position of the lattice
    # returns the transoformed spin chain
    spins_f = spin_chain.copy()
    # matrix multiplication only in the correct position of the basis vector
    spins_f[position] = list(np.matmul(current_operator,spin_chain[position]))
    spins_f[position+1] = list(np.matmul(next_operator,spin_chain[position+1]))    
    
    return spins_f

def calculate_matrix_term(spin_bra, spin_ket):
    # applies the heisenberg hamiltonian to each lattice site, for the hesinberg hamiltonian
    # we have 3 terms for each site.
    # args-> spin_bra: a list of lists representing the bra basis vector,
    #        spin_ket: a list of lists representing the ket basis vector
    # returns the matrix element as a number
    
    # save the spin chains just in case we need them latter
    sz_term = []
    # ket acted upon by first ladder operator product
    first_ladder = []
    # ket acted upon by second ladder operator product
    second_ladder = []
    eigen_values = []
    for i in range(0,len(spin_ket)-1):
        #Sz operator term
        transformed_spins = apply_operator(s_z,s_z,spin_ket,i)
        sz_term.append(transformed_spins)

        # First ladder operator term 
        transformed_spins = apply_operator(s_up,s_down,spin_ket,i)
        first_ladder.append(transformed_spins)

        # second ladder operator term
        transformed_spins = apply_operator(s_down,s_up,spin_ket,i)
        second_ladder.append(transformed_spins)

        # now we take the inner product with the basis Bra
        # to represent the inner product, sum the rows and then multiply all the elements to get the eigen value
        bracket = np.multiply(spin_bra,sz_term[i])
        eigen_values.append(np.prod(bracket.sum(1)))
        
        # remember that ladder operator terms have a 1/2 in front of them
        bracket = np.multiply(spin_bra,first_ladder[i])
        eigen_values.append(0.5*np.prod(bracket.sum(1)))

        bracket = np.multiply(spin_bra,second_ladder[i])
        eigen_values.append(0.5*np.prod(bracket.sum(1)))


    return np.sum(eigen_values)



Now we have to calculate all the matrix elements for the ground state part of the Hamiltonian. The ground state
is given by all the configurations corresponding to S=0

In [4]:
# Defining the ground state basis
basis_1 = [[1,0],[1,0], [0,1], [0,1]]
basis_2 = [[1,0],[0,1], [1,0], [0,1]]
basis_3 = [[1,0],[0,1], [0,1], [1,0]]
basis_4 = [[0,1],[1,0], [1,0], [0,1]]
basis_5 = [[0,1],[1,0], [0,1], [1,0]]
basis_6 = [[0,1],[0,1], [1,0], [1,0]]

basis_list = [ basis_1, basis_2, basis_3, basis_4, basis_5, basis_6]


In [5]:
# calculate each matrix element by iterating over the basis list two times
hamiltonian_matrix = np.zeros((len(basis_list), len(basis_list)))
# columns iteration
for i in range(len(basis_list)):
    # row iteration
    for j in range(len(basis_list)):
        hamiltonian_matrix[j,i] = calculate_matrix_term(basis_list[j],basis_list[i])

hamiltonian_matrix

array([[ 0.25,  0.5 ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.5 , -0.75,  0.5 ,  0.5 ,  0.  ,  0.  ],
       [ 0.  ,  0.5 , -0.25,  0.  ,  0.5 ,  0.  ],
       [ 0.  ,  0.5 ,  0.  , -0.25,  0.5 ,  0.  ],
       [ 0.  ,  0.  ,  0.5 ,  0.5 , -0.75,  0.5 ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.5 ,  0.25]])

In [6]:
# matrix diagonalization
# .eig returns a tuple of vectors
g_eigenvalues, g_eigenvectors = la.eig(np.asmatrix(hamiltonian_matrix))
# save the lowest eigenvalue and eigenvector which represent the ground state
ground_index = g_eigenvalues.argmin()

print("Eigenvalues for the groundstate block")
print(g_eigenvalues)
print()

# We now choose the lowest eigen value since it corresponds to the ground state
print("chosen eigenvalue")
print(g_eigenvalues[ground_index])
print()
print("chosen eigenvector")
print(g_eigenvectors[:,ground_index])
chosen_eigenvector = g_eigenvectors[:,ground_index]

Eigenvalues for the groundstate block
[-1.6160254 +0.j -0.95710678+0.j  0.1160254 +0.j  0.75      +0.j
  0.45710678+0.j -0.25      +0.j]

chosen eigenvalue
(-1.6160254037844373+0j)

chosen eigenvector
[ 0.14942925 -0.55767754  0.40824829  0.40824829 -0.55767754  0.14942925]


Matrix decomposition. We now calculate $\psi_{ij}$ for the matrix representation of the ground state.
Remember that, since ground state is always at S = 0, **the matrix elements that correspond to another value of the spin have to be set to zero**.

We have to choose how to sepparate our gorund state in two systems.

We will use the eigen vector with the highest eigen values, since the rest of them tend to be very small

## for now we do it by hand

# RESARCH PYTHON PERMUTATIONS

In [7]:
# define the two system basis
A_1 = [[1,0],[1,0]]
A_2 = [[1,0],[0,1]]
A_3 = [[0,1],[1,0]]
A_4 = [[0,1],[0,1]]

A_basis = [A_1,A_2,A_3,A_4]
B_basis = [A_1,A_2,A_3,A_4]


In [8]:
# we now calculate the ground state matrix representation
psi_ij = np.zeros((len(A_basis), len(B_basis)))
#rows
for i in range(0,len(A_basis)):
    #columns
    for j in range(0,len(B_basis)):
        is_in_gbasis = A_basis[j]+B_basis[i] in basis_list
        if is_in_gbasis == True: 
            # save index of the basis vector to find the eigen value
            eigen_index = basis_list.index(A_basis[j]+B_basis[i])
            psi_ij[i,j] = chosen_eigenvector[eigen_index]

rho_reduced = np.asmatrix(psi_ij)* np.asmatrix(psi_ij).H

In [9]:
np.set_printoptions(precision=7)
psi_ij

array([[ 0.       ,  0.       ,  0.       ,  0.1494292],
       [ 0.       , -0.5576775,  0.4082483,  0.       ],
       [ 0.       ,  0.4082483, -0.5576775,  0.       ],
       [ 0.1494292,  0.       ,  0.       ,  0.       ]])

In [10]:
rho_reduced

matrix([[ 0.0223291,  0.       ,  0.       ,  0.       ],
        [ 0.       ,  0.4776709, -0.4553418,  0.       ],
        [ 0.       , -0.4553418,  0.4776709,  0.       ],
        [ 0.       ,  0.       ,  0.       ,  0.0223291]])

Calculate the reduced density matrix via conjugate transposed

# 1.2 DMRG for particle in a box

### c) Exact Diagonalization of Disctretized Hamiltonian

### d) Density Matrix and Reduced density matrix for the ground state hamiltonian (Both Exact)

### e) DMRG


**Warmup Phase**

For a system with L sites, start by using a system of $L^´= 4$ and grow it until it reaches full size. Since this system has reflection 
symmetry we can take it up to $L^´= L/2$. Remember that the idea is to diagonalize 4x4 matrices always.



In [19]:
def update_hamiltonian(current_H, new_t12, new_h11):
    # updates the block hamiltonian to include the new matrix elements of the new basis
    # args---> current_H: Matrix representing the block hamiltonian from the last step
    #       new_t12, new_h11: scalats representing H11 and T12 matrix elements on the new basis, 
    # returns---> new_H: Matrix representing the new hamiltonian
    
    # create copy to make sure the memory is not the same
    new_H = current_H.copy()
    new_H[0,0] = new_h11
    new_H[0,1] = new_t12
    new_H[1,0] = new_t12
    # for the environment we have reflectin symmetry
    new_H[2,3] = new_t12
    new_H[3,2] = new_t12
    new_H[3,3] = new_h11
    
    return new_H


In [20]:
# System size definition
L = 40
# vector that saves effective hamiltonians
H_list = []
T_list = []

# initialize the hamiltonian for the warmup phase
H = np.asmatrix(np.zeros((4,4)))
np.fill_diagonal(H, 2)

for i in range(0,4-1):
    # fill upper diagonal
    H[i,i+1] = -1
    # fill lower diagonal
    H[i+1,i] = -1
type(H)

numpy.matrix

In [17]:
# initialize the blocks
H11 = 2.0
T12 = -1.0
T34 = -1.0
H44 = 2.0

In [13]:
# Form the hamiltonian for the current site
H[0,0] = H11
H[0,1] = T12
H[1,0] = T12
H[2,3] = T34
H[3,2] = T34
H[3,3] = H44
#save the hamiltonian for the current site
H_list.append(H)
T_list.append(T12)
#print(H_list[i])


In [14]:
for i in range(0,int(L/2)):
    # diagonalize and get the ground state energy and vector
    eigenvalues, eigenvectors = la.eig(np.asmatrix(H))
    g_index = eigenvalues.argmin()
    E_ground = eigenvalues[g_index]
    psi_ground = eigenvectors[g_index]
    
    # given that we have a left block the new basis is formed with the first two coefficients
    new_basis = np.asmatrix([psi_ground[0],psi_ground[1]])
    new_basis = 1/np.sqrt(psi_ground[0]**2 + psi_ground[1]**2) * new_basis
    
    a1 = new_basis[0,0]
    a2 = new_basis[0,1]

    # calculate the new block
    # this term is the updated wave function at site i+1
    T12 = - a2
    T34 = - a2
    H11 = a1**2*H11 + 2*a2**2 - 2*a1*a2*T12
    H44 = H11
    
    new_H = H.copy()
    new_H[0,0] = H11
    new_H[0,1] = T12
    new_H[1,0] = T12
    new_H[2,3] = T34
    new_H[3,2] = T34
    new_H[3,3] = H44
    H_list.append(new_H)

In [15]:
H_list

[matrix([[ 2., -1.,  0.,  0.],
         [-1.,  2., -1.,  0.],
         [ 0., -1.,  2., -1.],
         [ 0.,  0., -1.,  2.]]),
 matrix([[ 1.5297718, -0.5257311,  0.       ,  0.       ],
         [-0.5257311,  2.       , -1.       ,  0.       ],
         [ 0.       , -1.       ,  2.       , -0.5257311],
         [ 0.       ,  0.       , -0.5257311,  1.5297718]]),
 matrix([[ 1.1895115, -0.5257311,  0.       ,  0.       ],
         [-0.5257311,  2.       , -1.       ,  0.       ],
         [ 0.       , -1.       ,  2.       , -0.5257311],
         [ 0.       ,  0.       , -0.5257311,  1.1895115]]),
 matrix([[ 0.9432968, -0.5257311,  0.       ,  0.       ],
         [-0.5257311,  2.       , -1.       ,  0.       ],
         [ 0.       , -1.       ,  2.       , -0.5257311],
         [ 0.       ,  0.       , -0.5257311,  0.9432968]]),
 matrix([[ 0.7651342, -0.5257311,  0.       ,  0.       ],
         [-0.5257311,  2.       , -1.       ,  0.       ],
         [ 0.       , -1.       ,  2.     

First step